# Projeto de Sinais

## Sinais e Sistemas Dinâmicos - Prof. Derzu Omaia

## Jansepetrus Brasileiro Pereira e Nathália de Vasconcelos Silva

Descrição do projeto: https://www.dropbox.com/s/wzibx7g7136um4z/Projeto.pdf?dl=0

### Importar todas as bibliotecas necessárias para o projeto

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.fftpack import fft
from scipy.signal import butter, lfilter, freqz

### Definicao das funções de usuário a serem utilizadas

In [2]:
def plot_wav(wav, sample_rate, save_plot=False, nome_plot="", sufixo_nome=""):
    if(nome_plot == ""):
        from datetime import datetime
        nome_plot = str(datetime.now()).replace(":","-")
        
    times = np.arange(len(wav))/float(sample_rate)

    plt.figure(figsize=(30, 4))

    if(wav.ndim == 2):
        plt.fill_between(times, wav[:,0], wav[:,1], color='k') # para dual-channel
    else:
        plt.fill_between(times, wav)

    plt.xlim(times[0], times[-1])
    plt.xlabel('Tempo (s)')
    plt.ylabel('Amplitude')
    plt.title(sufixo_nome, loc='left')
    plt.title(nome_plot, loc='right')
    
    if(save_plot):
        plt.savefig(nome_plot + "_" + sufixo_nome + '.png', dpi=210)
    plt.show()

def plot_wav_freqDomain(fft_wav, freq, N, save_plot=False, nome_plot="", sufixo_nome=""):
    if(nome_plot == ""):
        from datetime import datetime
        nome_plot = str(datetime.now()).replace(":","-")
        
    plt.plot(freq, (1.0/N)*np.abs(fft_wav))
    plt.xlabel('Frequência (Hz)')
    plt.ylabel('Magnitude do Espectro (Domínio da Frequência)')
    plt.title(sufixo_nome, loc='left')
    plt.title(nome_plot, loc='right')
    plt.grid()
    if(save_plot):
        plt.savefig(nome_plot + "_" + sufixo_nome + '.png', dpi=210)
    plt.show()
    
def simple_plot_wav(wav_data, sample_rate, L, sufixo_nome="", save_plot=False, nome_plot=""):
    if(nome_plot == ""):
        from datetime import datetime
        nome_plot = str(datetime.now()).replace(":","-")

    plt.plot(np.arange(L) / sample_rate, wav_data)
    plt.xlim(0,L/sample_rate)
    plt.xlabel('Tempo (s)')
    plt.ylabel('Amplitude');
    plt.title(sufixo_nome, loc='left')
    plt.title(nome_plot, loc='right')
    
    if(save_plot):
        plt.savefig(nome_plot + "_" + sufixo_nome + '.png', dpi=210)
    plt.show()
    
def plot_resposta_freq(w, h, sample_rate, freq_corte, sufixo_nome="", save_plot=False, nome_plot=""):
    if(nome_plot == ""):
        from datetime import datetime
        nome_plot = str(datetime.now()).replace(":","-")
    
    #plt.subplot(2, 1, 1)
    plt.plot(0.5*sample_rate*w/np.pi, np.abs(h), 'b')
    plt.plot(freq_corte, 0.5*np.sqrt(2), 'ko')
    plt.axvline(freq_corte, color='k')
    plt.xlim(0, 0.5*sample_rate)
    plt.title("Resposta em Frequência - Filtro Passa-Baixa")
    plt.xlabel('Frequência (Hz)')
    plt.grid()
        
    if(save_plot):
        plt.savefig(nome_plot + "_" + sufixo_nome + '.png', dpi=210)
    
    plt.show()

### Definição dos Parâmetros de Execução

In [3]:
####
# Parametros de Execucao
####
PLOT_SINAL_ORIGINAL = False
PLOT_SINAL_FFT = False
PLOT_SINAL_FFT_SHIFT = False
PLOT_SINAL_FFT_SHIFT_RESPFREQ = False
PLOT_SINAL_FFT_SHIFT_BUTTER = False
PLOT_SINAL_FFT_SHIFT_BUTTER_COMPARATIVO = False
PLOT_SINAL_FFT_SHIFT_BUTTER_CORTE = False
MONO = True
MONO_ESQ = True
STEREO = not MONO
MONO_DIR = not MONO_ESQ

### Percorrer a pasta do dataset IRMAS, abrir todos os arquivos .wav e aplicar a transformada de Fourier

In [ ]:
#ROOT_PATH = 'IRMAS-TrainingData_red' + os.sep
ROOT_PATH = 'TESTE' + os.sep
#ROOT_PATH = 'UM' + os.sep

signal_dataset = []

for root, dirs, files in os.walk(ROOT_PATH):  
    for filename in files:
        ####
        # Leitura do arquivo WAV
        ####
        sample_rate, wav_data = wavfile.read(root + os.sep + filename)
        
        ####
        # Captura do sinal em um ndarray
        ####
        #if(wav_data.ndim == 2):
        #    if(STEREO):
        #        x = wav_data
        #if(MONO_DIR):
        #    x = wav_data[0:,1] # canal da direita
        #if(MONO_ESQ):
        #    x = wav_data[0:,0] # canal da esquerda
        wav_data = np.mean(wav_data, axis=1) #Conversao de sinal stereo para mono, fazendo a media dos canais esquerdo e direito
        
        ####
        # Especificacoes do dominio do tempo
        ####
        L = wav_data.shape[0]
        dt = 1.0 / sample_rate
        t = np.arange(0, (L/sample_rate)-dt, dt).T # Discretizacao da amostragem ao passo ((L/sample_rate)-dt)
        #t = np.linspace(0, dt, L)
        N = t.size + 1
        if (PLOT_SINAL_ORIGINAL):
            #plot_wav(wav_data, sample_rate, sufixo_nome="ORIG", save_plot=True, nome_plot=filename)
            simple_plot_wav(wav_data, sample_rate, L, sufixo_nome="_01_ORIG", save_plot=True, nome_plot=filename)
        
        ####
        # Especificacoes do dominio da frequencia
        ####
        dF = sample_rate/N # hertz
        freq = np.arange(-sample_rate/2, sample_rate/2-dF, dF)
        freq_max = np.amax(freq)
        
        ####
        # Aplicacao da FFT e Calculo da Frequencia do sinal
        ####
        fft_output = fft(wav_data) # Fourier.
        if (PLOT_SINAL_FFT):
            plot_wav_freqDomain(fft_output, freq, N, sufixo_nome="_02_FFT", save_plot=True, nome_plot=filename)

        ####
        # Movimentacao dos componentes de frequencia zero para o centro do espectro
        ####
        fft_shifted = np.fft.fftshift( fft_output ) # Desloca o centro zero da transformada para o centro.
        #freq_shifted = np.fft.fftshift(freq)
        
        if (PLOT_SINAL_FFT_SHIFT):
            plot_wav_freqDomain(fft_shifted, freq, N, sufixo_nome="_03_FFT-SHIFT", save_plot=True, nome_plot=filename)

        ####
        # Aplicacao do Filtro Passa-Baixa Butterworth
        ####
        butter_order = 6
        freq_corte = freq_max*0.3 # TODO: Precisa descobrir se essa e' uma boa frequencia de corte para utilizar no Passa-Baixa Butterworth
        butter_cutoff = freq_corte / (0.5 * sample_rate)
        
        b, a = butter(butter_order, butter_cutoff, btype='low', analog=False)
    
        w, h = freqz(b, a, worN=8000)
        
        if(PLOT_SINAL_FFT_SHIFT_RESPFREQ):
            plot_resposta_freq(w, h, sample_rate, freq_corte, sufixo_nome="_04_FFT-SHIFT-RESPFREQ", save_plot=True, nome_plot=filename)
        
        
        fft_X_filtro = lfilter(b, a, fft_shifted)
        
        if(PLOT_SINAL_FFT_SHIFT_BUTTER):
            plot_wav_freqDomain(fft_X_filtro, freq, N, sufixo_nome="_05_FFT-SHIFT-FILT", save_plot=True, nome_plot=filename)
        
        if(PLOT_SINAL_FFT_SHIFT_BUTTER_COMPARATIVO):
            plt.plot(freq, fft_shifted.real, 'b-', label='data')
            plt.plot(freq, fft_X_filtro.real, 'g-', linewidth=2, label='filtered data')
            plt.title("_06_FFT-SHIFT-BUTTER-COMPARATIVO", loc='left')
            plt.title(filename, loc='right')
            plt.xlabel('Time [sec]')
            plt.grid()
            plt.legend()
            plt.savefig(filename + "_" + "_06_FFT-SHIFT-BUTTER-COMPARATIVO" + '.png', dpi=210)
            plt.show()
        
        centro_espectro = (fft_X_filtro.shape[0])//2
        offset = int((fft_X_filtro.shape[0]) * 0.05) # 5% de offset a partir do centro do espectro
        
        fft_filtro_corte = fft_X_filtro[centro_espectro-offset:centro_espectro+offset,]
        freq_dataframe = freq[centro_espectro-offset:centro_espectro+offset,]

        if(PLOT_SINAL_FFT_SHIFT_BUTTER_CORTE):
            plot_wav_freqDomain(fft_filtro_corte, freq_dataframe, N, sufixo_nome="_07_FFT-CORTE", save_plot=True, nome_plot=filename)
        
        
        ####
        # Adicao do sinal filtrado ao DataSet, para ser transformado em um DataFrame.
        ####
        signal_dataset.append(fft_filtro_corte)

df_data = pd.DataFrame(signal_dataset)
DATAFRAME_NAME = 'WAV_DF_RED_20xInst.pkl'
df_data.to_pickle(DATAFRAME_NAME)

### Guardar o DataFrame no PC, para evitar executar o código acima

In [ ]:
DATAFRAME_NAME = 'WAV_DF.pkl'
df_data.to_pickle(DATAFRAME_NAME)

### Ler um DataFrame salvo na máquina

In [ ]:
DATAFRAME_NAME = 'WAV_DF.pkl'
data_frame = pd.read_pickle(DATAFRAME_NAME)

print(data_frame)
data_frame.describe()